In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import datetime as dt

from scipy import stats
from tempfile import mkdtemp
from shutil import rmtree

from xgboost import XGBRegressor

from sklearn import set_config
set_config(display = 'diagram')

# Sklearn preprocessing
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.ensemble import AdaBoostRegressor, VotingRegressor, GradientBoostingRegressor, StackingRegressor, RandomForestRegressor
from sklearn.feature_selection import SelectPercentile, mutual_info_regression, VarianceThreshold, SelectFromModel
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.metrics import make_scorer, mean_squared_error, mean_squared_log_error
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor

In [3]:
users_df = pd.read_csv('../raw_data/users.csv')
centers_df = pd.read_csv('../raw_data/distribution_centers.csv')
events_df = pd.read_csv('../raw_data/events.csv')
inventory_items_df = pd.read_csv('../raw_data/inventory_items.csv')
order_items_df = pd.read_csv('../raw_data/order_items.csv')
orders_df = pd.read_csv('../raw_data/orders.csv')
products_df = pd.read_csv('../raw_data/products.csv')

In [4]:
def data_cleaning(order_items_df, orders_df):
    '''
    This function takes the order_items and the orders tables, aggregates the order_items table,
    merges the tables, cleans the datatypes and returns the cleaned dataframe
    '''

    # Agreegate the sale_price by order_id and merge the orders table
    order_sales = order_items_df[['order_id','sale_price']].groupby(by='order_id').sum('sale_price').reset_index()

    cleaned_order_sales = orders_df.merge(order_sales, on='order_id', how='left')

    # Convert the created_at column from 'object' to datetime64
    cleaned_order_sales['created_at'] = pd.to_datetime(cleaned_order_sales['created_at'],
                                                       format='mixed').dt.tz_localize(None)
    cleaned_order_sales['returned_at'] = pd.to_datetime(cleaned_order_sales['returned_at'],
                                                       format='mixed').dt.tz_localize(None)
    cleaned_order_sales['shipped_at'] = pd.to_datetime(cleaned_order_sales['shipped_at'],
                                                       format='mixed').dt.tz_localize(None)
    cleaned_order_sales['delivered_at'] = pd.to_datetime(cleaned_order_sales['delivered_at'],
                                                       format='mixed').dt.tz_localize(None)


    return cleaned_order_sales

In [135]:
def order_features(orders_df, order_items_df, split_date):
    '''
    Calculates features required for XBG
    inputs order_df, order_items_df, products_df
    Assumes revenue per order is already calculated in sale_price column,
    all the datetime fields and split_date should be in datetime format
    '''

    orders_df = data_cleaning(order_items_df, orders_df)

    # Days calculation
    orders_df['days_to_split_date'] = (split_date - orders_df['created_at']).dt.days
    orders_df['days_delivery'] = (orders_df['delivered_at'] - orders_df['created_at']).dt.days

    # Filter data only for last 2 years
    orders_df = orders_df[(orders_df.days_to_split_date >= 0) & (orders_df.days_to_split_date < 365*2)]

    # Base dataset : user level
    base = orders_df[['user_id','order_id']].groupby('user_id').count().rename(columns={'order_id': 'orders_total'})

    # Calculate first purchase, latest purchase
    first_purchase = orders_df[['user_id','created_at']].groupby('user_id').min().rename(columns={'created_at': 'min_created_at'})
    base = base.merge(first_purchase, on='user_id')
    latest_purchase = orders_df[['user_id','created_at']].groupby('user_id').max().rename(columns={'created_at': 'max_created_at'})
    base = base.merge(latest_purchase, on='user_id')


    # Days from first/last order (Recency, customer T)
    base['days_from_first_order'] =  (split_date - base['min_created_at']).dt.days
    base['days_from_last_order'] =  (split_date - base['max_created_at']).dt.days


    # Frequency, Monetary
    df_30 = orders_df[orders_df.days_to_split_date < 30]
    df_60 = orders_df[(orders_df.days_to_split_date >= 30) & (orders_df.days_to_split_date < 60)]
    df_120 = orders_df[(orders_df.days_to_split_date >= 60) & (orders_df.days_to_split_date < 120)]
    df_240 = orders_df[(orders_df.days_to_split_date >= 120) & (orders_df.days_to_split_date < 240)]
    df_480 = orders_df[(orders_df.days_to_split_date >= 240) & (orders_df.days_to_split_date < 480)]

    # orders per different timeframe
    f_30 = df_30[['user_id','order_id']].groupby(by='user_id').count().fillna(0).rename(columns={'order_id': 'orders_30d'})
    f_60 = df_60[['user_id','order_id']].groupby(by='user_id').count().fillna(0).rename(columns={'order_id': 'orders_60d'})
    f_120 = df_120[['user_id','order_id']].groupby(by='user_id').count().fillna(0).rename(columns={'order_id': 'orders_120d'})
    f_240 = df_240[['user_id','order_id']].groupby(by='user_id').count().fillna(0).rename(columns={'order_id': 'orders_240d'})
    f_480 = df_480[['user_id','order_id']].groupby(by='user_id').count().fillna(0).rename(columns={'order_id': 'orders_480d'})

    # Frequency Joined to base dataset
    base = base.merge(f_30,on='user_id',how='left')
    base = base.merge(f_60,on='user_id',how='left')
    base = base.merge(f_120,on='user_id',how='left')
    base = base.merge(f_240,on='user_id',how='left')
    base = base.merge(f_480,on='user_id',how='left')

    # orders per different timeframe
    m_30 = df_30[['user_id','sale_price']].groupby(by='user_id').sum().fillna(0).rename(columns={'sale_price': 'revenue_30d'})
    m_60 = df_60[['user_id','sale_price']].groupby(by='user_id').sum().fillna(0).rename(columns={'sale_price': 'revenue_60d'})
    m_120 = df_120[['user_id','sale_price']].groupby(by='user_id').sum().fillna(0).rename(columns={'sale_price': 'revenue_120d'})
    m_240 = df_240[['user_id','sale_price']].groupby(by='user_id').sum().fillna(0).rename(columns={'sale_price': 'revenue_240d'})
    m_480 = df_480[['user_id','sale_price']].groupby(by='user_id').sum().fillna(0).rename(columns={'sale_price': 'revenue_480d'})
    m_total = orders_df[['user_id','sale_price']].groupby(by='user_id').sum().fillna(0).rename(columns={'sale_price': 'revenue_total'})

    # Monetary Joined to base dataset
    base = base.merge(m_30,on='user_id',how='left')
    base = base.merge(m_60,on='user_id',how='left')
    base = base.merge(m_120,on='user_id',how='left')
    base = base.merge(m_240,on='user_id',how='left')
    base = base.merge(m_480,on='user_id',how='left')
    base = base.merge(m_total,on='user_id',how='left')

    # ABV calculation
    base['abv'] = base['revenue_total']/base['orders_total']

    # Other calcs (delivery, num_items)
    avg_delivery_days = orders_df[['user_id','days_delivery']].groupby('user_id').mean().rename(columns={'days_delivery': 'avg_delivery_days'})
    base = base.merge(avg_delivery_days, on='user_id')
    avg_items_p_order = orders_df[['user_id','num_of_item']].groupby('user_id').mean().rename(columns={'num_of_item': 'avg_num_items'})
    base = base.merge(avg_items_p_order, on='user_id')

    return base

In [6]:
temp_date = dt.datetime(2020,1,1)
result = order_features(temp_date, orders_df, order_items_df)
result.head(5)

,orders_total,min_created_at,max_created_at,days_from_first_order,days_from_last_order,orders_30d,orders_60d,orders_120d,orders_240d,orders_480d,revenue_30d,revenue_60d,revenue_120d,revenue_240d,revenue_480d,revenue_total,abv,avg_delivery_days,avg_num_items
user_id,,,,,,,,,,,,,,,,,,,
17,1,2019-06-15 12:17:00,2019-06-15 12:17:00,199,199,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,173.959999,NaN,173.959999,173.959999,1.0,2.0
115,1,2019-09-04 14:50:00,2019-09-04 14:50:00,118,118,NaN,NaN,1.0,NaN,NaN,NaN,NaN,23.950001,NaN,NaN,23.950001,23.950001,NaN,1.0
158,1,2019-04-16 11:12:00,2019-04-16 11:12:00,259,259,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,66.989998,66.989998,66.989998,NaN,1.0
178,1,2019-12-20 04:20:00,2019-12-20 04:20:00,11,11,1.0,NaN,NaN,NaN,NaN,59.990002,NaN,NaN,NaN,NaN,59.990002,59.990002,NaN,1.0
273,1,2019-12-10 11:53:00,2019-12-10 11:53:00,21,21,1.0,NaN,NaN,NaN,NaN,38.360001,NaN,NaN,NaN,NaN,38.360001,38.360001,5.0,1.0


In [40]:
products_df.columns

Index(['id', 'cost', 'category', 'name', 'brand', 'retail_price', 'department',
       'sku', 'distribution_center_id'],
      dtype='object')

In [102]:
def join_tables_item_level(order_items_df, products_df):
    '''
    Joins order-item & product
    '''
    order_items_df = order_items_df[['id', 
                                     'order_id', 
                                     'user_id', 
                                     'product_id', 
                                     'inventory_item_id',
                                     # 'status', 
                                     'created_at', 
                                     # 'shipped_at', 
                                     # 'delivered_at', 
                                     # 'returned_at',
                                     'sale_price']].rename(columns={'id':'order_item_id'})

    order_items_df['created_at'] = pd.to_datetime(order_items_df['created_at'],
                                                       format='mixed').dt.tz_localize(None)
    products_df = products_df[['id', 
                               'cost', 
                               'category', 
                               'name',
                               'brand',
                               'retail_price', 
                               # 'department',
                               # 'sku',
                               'distribution_center_id']]

    
    df_joined = order_items_df.merge(products_df, left_on='product_id', right_on='id', how='left')
    df_joined = df_joined.drop(columns=['id']) # duplicate product id    

    return df_joined

In [74]:
item_level_df = join_tables_item_level(order_items_df, products_df)

In [85]:
item_level_df[['user_id','brand','sale_price']].groupby(by=['user_id','brand']).sum()

sale_price
user_id brand                      
1       Joe's Jeans      125.000000
        Tommy Hilfiger    15.000000
        eVogues Apparel   19.990000
2       Tommy Bahama      22.000000
3       Dockers           59.990002
...                             ...
99998   Woolrich          55.000000
99999   Hanes             25.000000
        Wrangler          42.980000
100000  Jockey            22.500000
        Paddi Murphy     129.949997

[180443 rows x 1 columns]

In [183]:
def calc_product_features(item_level_df, split_date):
    '''
    takes item-level dataframe and calculate features to user level.
    '''

    # filter data only for last 2 years
    min_date = split_date - pd.DateOffset(days=365*2)
    item_level_df = item_level_df[(item_level_df['created_at'] <= split_date) & (item_level_df['created_at'] > min_date)].copy()
    
    # Calculate the first orders
    item_level_df['order_rank'] = item_level_df.groupby('user_id')['created_at'].rank().astype(int) # calculating the order of orders
    item_level_df['order_rank_desc'] = item_level_df.groupby('user_id')['created_at'].rank(method='max').astype(int) # calculating the order of orders

    base = item_level_df[['user_id','order_item_id']].groupby('user_id').count().fillna(0).rename(columns={'order_item_id':'num_order_item'})
    
    # taking value for the very first order
    intial_values = item_level_df[item_level_df['order_rank']==1][['user_id','category','brand']].rename(columns={'category':'categ_initial',
                                                                                                                  'brand':'brand_initial'})
    base = base.merge(intial_values, on='user_id', how='left')

    # taking value for the very last order
    last_values = item_level_df[item_level_df['order_rank_desc']==1][['user_id','category','brand']].rename(columns={'category':'categ_last',
                                                                                                                  'brand':'brand_last'})
    base = base.merge(last_values, on='user_id', how='left')

    item_level_df = item_level_df.rename(columns={'created_at':'order_item_created_at'})

    # Most frequent brand and category (later)
    # mode_values = item_level_df[item_level_df['order_rank']==1][['user_id','category','brand']].rename(columns={'category':'categ_initial',
                                                                                                                  # 'brand':'brand_initial'})
    return base

In [179]:
item_level_df.columns

Index(['order_item_id', 'order_id', 'user_id', 'product_id',
       'inventory_item_id', 'created_at', 'sale_price', 'cost', 'category',
       'name', 'brand', 'retail_price', 'distribution_center_id'],
      dtype='object')

In [184]:
calc_product_features(item_level_df, temp_date).head()

,user_id,num_order_item,categ_initial,brand_initial,categ_last,brand_last
0,17,2,Outerwear & Coats,Tommy Hilfiger,Outerwear & Coats,Tommy Hilfiger
1,115,1,Fashion Hoodies & Sweatshirts,FineBrandShop,Fashion Hoodies & Sweatshirts,FineBrandShop
2,158,1,Suits,Ice,Suits,Ice
3,178,1,Shorts,Dickies,Shorts,Dickies
4,273,1,Skirts,Jones New York,Skirts,Jones New York


In [143]:
def select_user_columns(users_df, split_date):

    df = users_df[['id','age','country','created_at']].rename(columns={'id':'user_id','created_at':'user_created_at'})
    # Filter users only for last 2 years from split date
    min_date = split_date - pd.DateOffset(days=365*2)
    df['user_created_at'] = pd.to_datetime(df['user_created_at'], format='mixed').dt.tz_localize(None)
    df = df[(df['user_created_at'] <= split_date) & (df['user_created_at'] > min_date)]
    
    return df

In [147]:
item_level_df.columns

Index(['order_item_id', 'order_id', 'user_id', 'product_id',
       'inventory_item_id', 'created_at', 'sale_price', 'cost', 'category',
       'name', 'brand', 'retail_price', 'distribution_center_id'],
      dtype='object')

In [166]:
def create_y_actual(order_items_df, split_date):
    df = order_items_df[['id','user_id','created_at','sale_price']].copy().rename(columns={'id':'order_item_id'})
    df['order_created_at'] = pd.to_datetime(df['created_at'], format='mixed').dt.tz_localize(None)

    # select next 90 days of revenue from split_date
    max_date = split_date + pd.DateOffset(days=90)
    df = df[(df['order_created_at'] <= max_date) & (df['order_created_at'] > split_date)]
    
    df_agg = df[['user_id','sale_price']].groupby('user_id').sum().fillna(0).rename(columns={'sale_price':'revenue_next_90d'})

    return df_agg

In [185]:
# Setting test dataset

test_date = dt.datetime(2020,1,1)
item_level_df = join_tables_item_level(order_items_df, products_df)

In [186]:
df_user = select_user_columns(users_df, test_date)
df_order = order_features(orders_df, order_items_df, test_date)
df_item = calc_product_features(item_level_df, test_date)
df_y_actual = create_y_actual(order_items_df, test_date)

In [187]:
df_item.columns

Index(['user_id', 'num_order_item', 'categ_initial', 'brand_initial',
       'categ_last', 'brand_last'],
      dtype='object')

In [198]:
def prep_input_dataset(df_user, df_order, df_item, df_y_actual):
    '''
    joins all preprocessed user-level-df
    '''
    
    final_df = df_user.merge(df_order, on='user_id', how='left')
    final_df = final_df.merge(df_item, on='user_id', how='left')
    final_df = final_df.merge(df_y_actual, on='user_id', how='left')

    final_df = final_df[final_df['orders_total']>0].copy()

    final_df = final_df.reset_index()

    return final_df


In [199]:
input_dataset = prep_input_dataset(df_user, df_order, df_item, df_y_actual)
input_dataset.columns

Index(['index', 'user_id', 'age', 'country', 'user_created_at', 'orders_total',
       'min_created_at', 'max_created_at', 'days_from_first_order',
       'days_from_last_order', 'orders_30d', 'orders_60d', 'orders_120d',
       'orders_240d', 'orders_480d', 'revenue_30d', 'revenue_60d',
       'revenue_120d', 'revenue_240d', 'revenue_480d', 'revenue_total', 'abv',
       'avg_delivery_days', 'avg_num_items', 'num_order_item', 'categ_initial',
       'brand_initial', 'categ_last', 'brand_last', 'revenue_next_90d'],
      dtype='object')

In [189]:
input_dataset[['user_id','user_created_at','orders_total','days_from_first_order','revenue_total','abv', 'revenue_next_90d']].head(20)

,user_id,user_created_at,orders_total,days_from_first_order,revenue_total,abv,revenue_next_90d
7,38813,2019-06-06 15:38:00,1.0,134.0,34.310001,34.310001,NaN
13,2392,2019-01-10 05:12:00,1.0,96.0,39.990002,39.990002,NaN
18,62726,2019-10-17 13:39:00,1.0,9.0,27.950001,27.950001,NaN
21,49899,2019-04-04 07:37:00,1.0,130.0,34.990002,34.990002,NaN
28,14802,2019-02-05 10:04:00,1.0,183.0,156.490000,156.490000,NaN
30,88924,2019-02-10 17:46:00,1.0,199.0,69.500000,69.500000,NaN
44,49697,2019-10-31 00:01:00,1.0,41.0,32.000000,32.000000,NaN
52,6611,2019-04-11 03:29:00,1.0,235.0,49.000000,49.000000,NaN
56,56020,2019-12-19 06:07:00,1.0,3.0,98.870003,98.870003,39.880001
69,54562,2019-01-29 02:53:00,1.0,85.0,56.920000,56.920000,NaN


In [190]:
input_dataset.dtypes

user_id                           int64
age                               int64
country                          object
user_created_at          datetime64[ns]
orders_total                    float64
min_created_at           datetime64[ns]
max_created_at           datetime64[ns]
days_from_first_order           float64
days_from_last_order            float64
orders_30d                      float64
orders_60d                      float64
orders_120d                     float64
orders_240d                     float64
orders_480d                     float64
revenue_30d                     float64
revenue_60d                     float64
revenue_120d                    float64
revenue_240d                    float64
revenue_480d                    float64
revenue_total                   float64
abv                             float64
avg_delivery_days               float64
avg_num_items                   float64
num_order_item                  float64
categ_initial                    object


In [200]:
used_columns = [
    'user_id',
    'age',
    'country',
    'orders_total',
    'days_from_first_order',
    'days_from_last_order',
    'orders_30d',
    'orders_60d',
    'orders_120d',
    'orders_240d',
    'orders_480d',
    'revenue_30d',
    'revenue_60d',
    'revenue_120d',
    'revenue_240d',
    'revenue_480d',
    'revenue_total',
    'abv',
    'avg_delivery_days',
    'avg_num_items',
    'num_order_item',
    'categ_initial',
    'brand_initial',
    'revenue_next_90d'
    ]

In [201]:
X = input_dataset[used_columns].drop(columns=['revenue_next_90d']).copy()
y = input_dataset[['user_id','revenue_next_90d']].copy()

In [207]:
X.head(10)

,user_id,age,country,orders_total,days_from_first_order,days_from_last_order,orders_30d,orders_60d,orders_120d,orders_240d,...,revenue_120d,revenue_240d,revenue_480d,revenue_total,abv,avg_delivery_days,avg_num_items,num_order_item,categ_initial,brand_initial
0,38813,70,Brasil,1.0,134.0,134.0,NaN,NaN,NaN,1.0,...,NaN,34.310001,NaN,34.310001,34.310001,5.0,1.0,1.0,Shorts,Cubavera
1,2392,59,Japan,1.0,96.0,96.0,NaN,NaN,1.0,NaN,...,39.990002,NaN,NaN,39.990002,39.990002,NaN,1.0,1.0,Outerwear & Coats,Southpole
2,62726,23,Japan,1.0,9.0,9.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,27.950001,27.950001,4.0,1.0,1.0,Tops & Tees,Eddie Bauer
3,49899,51,Japan,1.0,130.0,130.0,NaN,NaN,NaN,1.0,...,NaN,34.990002,NaN,34.990002,34.990002,NaN,1.0,1.0,Active,Guide Gear
4,14802,53,Japan,1.0,183.0,183.0,NaN,NaN,NaN,1.0,...,NaN,156.490000,NaN,156.490000,156.490000,NaN,4.0,4.0,Swim,RVCA
5,88924,38,United States,1.0,199.0,199.0,NaN,NaN,NaN,1.0,...,NaN,69.500000,NaN,69.500000,69.500000,3.0,1.0,1.0,Jeans,Quiksilver
6,49697,44,United States,1.0,41.0,41.0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,32.000000,32.000000,NaN,1.0,1.0,Sleep & Lounge,Harbor Bay
7,6611,19,United States,1.0,235.0,235.0,NaN,NaN,NaN,1.0,...,NaN,49.000000,NaN,49.000000,49.000000,NaN,1.0,1.0,Intimates,Jezebel
8,56020,59,United States,1.0,3.0,3.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,98.870003,98.870003,NaN,3.0,2.0,Underwear,Papi
9,54562,30,Brasil,1.0,85.0,85.0,NaN,NaN,1.0,NaN,...,56.920000,NaN,NaN,56.920000,56.920000,NaN,2.0,2.0,Sleep & Lounge,Nautica


In [203]:
columns_to_make_zero = used_columns = [
    'orders_total',
    'orders_30d',
    'orders_60d',
    'orders_120d',
    'orders_240d',
    'orders_480d',
    'revenue_30d',
    'revenue_60d',
    'revenue_120d',
    'revenue_240d',
    'revenue_480d',
    'revenue_total',
    'num_order_item',
    'revenue_next_90d'
    ]

In [206]:
# making NaN to 0
input_dataset[columns_to_make_zero].isna().sum()


orders_total           0
orders_30d          1945
orders_60d          2040
orders_120d         1736
orders_240d         1598
orders_480d         2114
revenue_30d         1945
revenue_60d         2040
revenue_120d        1736
revenue_240d        1598
revenue_480d        2114
revenue_total          0
num_order_item         6
revenue_next_90d    2240
dtype: int64